In [7]:
import json
path = "Apps_for_Android_5.json"
dict_edge = {} #example: 8842281e1d1347389f2ab93d60773d4d|23310161 : One of my favorite books.
dict_num_to_id = {} # reorder the node's id
edge_score = []
count = 0
review_text = "Reviewer [reviewerName] left a review on [reviewTime], giving the product [rating] stars. In his/her review, he/she wrote: [reviewText]. His/Her summary was [summary]."
with open(path) as f:
    for line in f:
        d = json.loads(line)
        edge = d["reviewerID"] + "|" + d["asin"]
        try:
            reviewtext = review_text.replace("[reviewerName]", d["reviewerName"])
        except:
            reviewtext = review_text.replace("[reviewerName]", "")
        if d["reviewTime"] == "":
            reviewtext = reviewtext.replace("[reviewTime]", "Unknown reviewtime")
        else:
            reviewtext = reviewtext.replace("[reviewTime]", d["reviewTime"])
        if d["overall"] == "":
            reviewtext = reviewtext.replace("[rating]", "Unknown")
        else:
            reviewtext = reviewtext.replace("[rating]", str(d["overall"]))
        reviewtext = reviewtext.replace("[reviewText]", d["reviewText"])
        if d["summary"] == "":
            reviewtext = reviewtext.replace("[summary]", "Unknown")
        else:
            reviewtext = reviewtext.replace("[summary]", d["summary"])
        dict_edge[edge] = reviewtext
        edge_score.append(d["overall"])
        if d["reviewerID"] not in dict_num_to_id:
            dict_num_to_id[d["reviewerID"]] = count
            count += 1
        if d["asin"] not in dict_num_to_id:
            dict_num_to_id[d["asin"]] = count
            count += 1
            

In [8]:
import json
dict_id_to_text = {}
dictid_to_label = {}
node_texts = "item"

with open("meta_Apps_for_Android.json") as f:
    for line in f:
        d = json.loads(line)
        break

In [10]:
import json
dict_id_to_text = {}
dictid_to_label = {}
node_texts = "item"

with open("meta_Apps_for_Android.json") as f:
    for line in f:
        d = json.loads(line)
        label_list = []
        for x in d["categories"]:
            for label in x:
                label_list.append(label)
        dictid_to_label[d["asin"]] = label_list
        product_text = node_texts
        '''
        product_text = nodes_texts.replace("[title]", d["title"])
        category_text = ", ".join(label_list[1:])
        product_text = product_text.replace("[category]", category_text)
        if d["feature"] == []:
            product_text = product_text.replace("[feature]","Unknown feature")
        else:
            feature_text = ", ".join(d["feature"])
            product_text = product_text.replace("[feature]",feature_text)
        if d["description"] == []:
            product_text = product_text.replace("[description]","Unknown description")
        else:
            description_text = ", ".join(d["description"])
            product_text = product_text.replace("[description]",description_text)
        if d["fit"] == "":
            product_text = product_text.replace("[fit]","Unknown fit")
        else:
            product_text = product_text.replace("[fit]",d["fit"])
        if d["price"] == "":
            product_text = product_text.replace("[price]","Unknown price")
        else:
            product_text = product_text.replace("[price]",d["price"])
        if d["brand"] == "":
            product_text = product_text.replace("[brand]","Unknown brand")
        else:
            product_text = product_text.replace("[brand]",d["brand"])
        if d["rank"] == "":
            product_text = product_text.replace("[rank]","Unknown rank")
        else:
            try:
                product_text = product_text.replace("[rank]",d["rank"])
            except:
                product_text = product_text.replace("[rank]","Unknown rank")
        if d["date"] == "":
            product_text = product_text.replace("[date]","Unknown date")
        else:
            product_text = product_text.replace("[date]",d["date"])
        if d["imageURL"] == []:
            product_text = product_text.replace("[imageURL]","Unknown imageURL")
        else:
            imageURL_text = ", ".join(d["imageURL"])
            product_text = product_text.replace("[imageURL]",imageURL_text)
        if d["imageURLHighRes"] == []:
            product_text = product_text.replace("[imageURLHighRes]","Unknown imageURLHighRes")
        else:
            imageURLHighRes_text = ", ".join(d["imageURLHighRes"])
            product_text = product_text.replace("[imageURLHighRes]",imageURLHighRes_text)
        '''
        dict_id_to_text[d["asin"]] = product_text

In [11]:
edge1 = [] 
edge2 = []  # edge1 edge2 are to generate edge_index
text_nodes = [None] * len(dict_num_to_id)
text_edges = []
text_node_labels = [-1] * len(dict_num_to_id)

In [12]:
for edge, edge_text in dict_edge.items():
    node1 = edge.split("|")[0]
    node2 = edge.split("|")[1]
    node1_id = int(dict_num_to_id[node1])
    node2_id = int(dict_num_to_id[node2])
    edge1.append(node1_id)
    edge2.append(node2_id)
    text_nodes[node1_id] = "reviewer"
    try:
        text_nodes[node2_id] = dict_id_to_text[node2]
    except:
        text_nodes[node2_id] = "Unknown node texts"
    text_edges.append(edge_text)
    try:
        text_node_labels[node2_id] = dictid_to_label[node2]
    except:
        text_node_labels[node2_id] = -1

In [14]:
from torch_geometric.data import Data
import torch

In [15]:
edge_index = torch.tensor([edge1,edge2])

In [16]:
new_data = Data(
    edge_index=edge_index,
    text_nodes=text_nodes,
    text_edges=text_edges,
    text_node_labels=text_node_labels,
    edge_score=edge_score
)

In [17]:
import pickle
output_file_path = '../processed/apps.pkl'
with open(output_file_path, 'wb') as output_file:
    pickle.dump(new_data, output_file)

print(f"Data saved to {output_file_path}")

Data saved to ../processed/apps.pkl
